# Paramerterized SQL Queries

In [1]:
import delta
import pyspark.sql.functions as F
import pathlib
import pyspark

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = delta.configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19a9be01-c471-42d8-b49a-e1c906b2babc;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 105ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default    

In [4]:
spark.read.format("delta").load(f"{pathlib.Path.home()}/data/delta/G1_1e7_1e2_0_0").createOrReplaceTempView("h20_1e7")

In [5]:
spark.sql("select * from h20_1e7 limit 3").show()

23/07/31 09:20:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+-----+------------+---+---+-----+---+---+---------+
|  id1|  id2|         id3|id4|id5|  id6| v1| v2|       v3|
+-----+-----+------------+---+---+-----+---+---+---------+
|id008|id052|id0000073659| 84| 89|82005|  5| 11|64.785802|
|id079|id037|id0000041462|  4| 35|28153|  1|  1|28.732545|
|id098|id031|id0000027269| 27| 38|13508|  5|  2|59.867875|
+-----+-----+------------+---+---+-----+---+---+---------+



In [6]:
spark.sql("SELECT id1, sum(v1) AS v1 from h20_1e7 GROUP BY id1").show()

+-----+------+
|  id1|    v1|
+-----+------+
|id089|300446|
|id080|302108|
|id087|301327|
|id073|298940|
|id043|299557|
|id064|297115|
|id051|299952|
|id045|298932|
|id074|301660|
|id023|299417|
|id006|299453|
|id013|300554|
|id055|298464|
|id099|301060|
|id056|299675|
|id052|299704|
|id093|298750|
|id075|299995|
|id034|297988|
|id036|302772|
+-----+------+
only showing top 20 rows



In [13]:
spark.sql("SELECT id1, sum(v1) AS v1 from {table_name} GROUP BY id1", table_name="h20_1e7").show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near ''h20_1e7'': extra input ''h20_1e7''.(line 1, pos 31)

== SQL ==
SELECT id1, sum(v1) AS v1 from 'h20_1e7' GROUP BY id1
-------------------------------^^^


In [22]:
spark.sql("SELECT id1, sum(v1) AS v1 from h20_1e7 GROUP BY % {groupby_col_name}", args={"groupby_col_name": "id1"}).show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '%'.(line 1, pos 48)

== SQL ==
SELECT id1, sum(v1) AS v1 from h20_1e7 GROUP BY % {groupby_col_name}
------------------------------------------------^^^


In [14]:
mydf = spark.range(10)

In [15]:
spark.sql("SELECT id FROM {mydf} WHERE id % @param1 = 0", mydf=mydf, param1='3').show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '@': extra input '@'.(line 1, pos 68)

== SQL ==
SELECT id FROM _pyspark_887b220d8455457e80f3d0f8ac9ab6f9 WHERE id % @param1 = 0
--------------------------------------------------------------------^^^
